In [ ]:
import pandas as pd
import itertools
import os
import gzip

import similarityregression.PairwiseAlignment as pwsaln
import similarityregression.AlignmentTools as alntools
import similarityregression.PredictSimilarity as srpred

# Introduction

This notebook describes an example of how to parse protein sequences from CisBP into a form that can be used for Similarity Regression (training and scoring). 

# Parse CisBP Data
Source data: 5 Homeodomain PBM experiments from CisBP (http://cisbp.ccbr.utoronto.ca/)

## Read E-scores and calculate motif similarity (E-Score Overlaps)

In [ ]:
escores = pd.read_csv('HomeodomainData/EScore.txt', index_col=0, delimiter='\t')
escores.columns = [x.split(':')[0] for x in escores.columns] #Relabel by moits
escores = escores.groupby(by=escores.columns, axis=1).mean() #Average E-scores for replicates

#Calculate E-score Overlaps
Escorethresh = 0.45
EScoreOverlaps = {}

MIDs = list(escores.columns)
MIDs.sort()
for x,y in itertools.combinations(MIDs, 2):
    #print x, y
    maxN = max(len(set(escores[escores[x] >= Escorethresh][x].index)), len(set(escores[escores[y] >= Escorethresh][y].index)))
    x_maxN = set(escores[x].sort_values(ascending = False).iloc[:maxN].index)
    y_maxN = set(escores[y].sort_values(ascending = False).iloc[:maxN].index)
    escoreoverlap = 1.0*len(x_maxN.intersection(y_maxN))/maxN
    #print escoreoverlap, len(x_maxN.intersection(y_maxN)), maxN
    EScoreOverlaps[(x,y)] = escoreoverlap

## TF / Protein Information 
### Parse Protein Information to reference DBD alignment

In [ ]:
proteins = pd.read_csv('HomeodomainData/prot_seq.txt', index_col=0, delimiter='\t')

motifs = []
with open('HomeodomainData/TF_Information.txt', 'r') as infile:
    h = infile.readline().strip().split('\t')
    for line in infile:
        line = line.strip().split('\t')
        data = line[:6]
        mids = line[6:]
        mids = [x[:-1] for x in mids]
        for mid in mids:
            motifs.append([mid] + data)
motifs = pd.DataFrame(motifs, columns=[h[-1]] + h[:-1])
motifs.set_index('Motif_ID', inplace=True)

In [ ]:
DBDseqs = set()
for x in proteins['DBD_seqs']:
    x = x.split(',') #in cases of multiple DBDs
    for seq in x:
        DBDseqs.add(seq)
        
#Write DBDseqs to fasta
with open('HDSeqs.fa', 'w') as outfile:
    for DBDseq in DBDseqs:
        outfile.write('>' + DBDseq + '\n' + DBDseq + '\n')

### Align to Pfam HMM

Source: [https://www.ebi.ac.uk/interpro/entry/pfam/PF00046](https://web.archive.org/web/20190619115817/https://pfam.xfam.org/family/PF00046/hmm) (Wayback Machine link)

```bash
# from within the `Examples` subdirectory
RunAPHID.py HomeodomainData/Homeodomain.hmm HDSeqs.fa semiglobal
```

In [ ]:
#Read results into dictionary that maps the DBD sequence to its reference alignment
DBDseqs = {} #Actual DBDseq : Reference Alignment 
for seq, aln in alntools.FastaIter(fileloc='DBDMatchPos_aphid/HDSeqs.matchpos_semiglobal.fa'):
    DBDseqs[seq] = aln

### Get Motif Sequences and Alignments to Reference

In [ ]:
MotifSequences = {} # MID: [DBD Sequences]
for mid in MIDs:
    #print mid
    minfo = motifs.loc[mid]
    protinfo = proteins.loc[minfo['TF_ID']]
    if type(protinfo['DBD_seqs']) == str:
        MotifSequences[mid] = protinfo['DBD_seqs'].split(',')
    else:
        MotifSequences[mid] = protinfo['DBD_seqs'][0].split(',')
    
#Get Motifs Sequences that are aligned to common Pfam reference
MotifSequences_aligned = {}
for mid, unaligned in MotifSequences.items():
    MotifSequences_aligned[mid] = [DBDseqs.get(x) for x in unaligned]

## Make DBD alignments and positional features for motif pairs with E-score overlaps

- The MotifAlignments contain the features, and outputs necessary to output into dataframes for training in R. 
- They can also be scored using existing SR models

In [ ]:
Homeodomain_ReplicateThreshold = 0.5555555

MotifAlignments = {} # MID Pair : {Alignment Information}

for pair, overlap in EScoreOverlaps.items():
    #Get Reference alignments for each motif
    proteinseqs_x = (pair[0], MotifSequences_aligned[pair[0]])
    proteinseqs_y = (pair[1], MotifSequences_aligned[pair[1]])
    #Align each motif to eachother
    sr_alignment = pwsaln.AlignDBDArrays(proteinseqs_x, proteinseqs_y)
    #Add E-score overlap to the alignment information
    sr_alignment['EScoreOverlap'] = overlap
    sr_alignment['EClass'] = int(overlap >= Homeodomain_ReplicateThreshold)
    MotifAlignments[pair] = sr_alignment

### Examples of information in an SR alignment 
Example: 

M1009_1.02:Berger08:Hoxa7_2668 vs. M1072_1.02:Badis09:Hoxa3_2783

In [ ]:
sr_alignment = MotifAlignments[('M1009_1.02', 'M1072_1.02')]

print 'Motif Similarity (E-score Overlap):', sr_alignment['EScoreOverlap']
print 'Alignment % Amino Acid Identity:', '{:.3f}'.format(100*sr_alignment['PctID_L']) + '%'
print 'Positional Amino Acid Identity:', sr_alignment['ByPos.PctID']
print 'Positional Amino Acid Similarity:', sr_alignment['ByPos.AvgB62']

# Using the SR Alignment dictionary to make SR training data (A), or score TF pairs using existing models (B)

## (A) Example of writing the SR alignment dictionary to files/directories for model training with the Rscript (SimilarityRegression_Train.R)

This is a  streamlined function to output the training data necessary for the Rscript. The code in "`Create DBD alignments and training dataframes.ipynb`" further parses heldout data, and can also handle constructs that are exempt from testing.

In [ ]:
def CreateSRTrainingFrames(motifalignments, loc_OutputFiles = 'sr_out/'):
        #Set up lists of Motif IDs. These are used to make the leave-one-out CV folds
        uIDs = set()
        IDs = []
        count = 0
    
        #Open Outputs (and make folders if necessary)
        if os.path.isdir(loc_OutputFiles) == False:
            os.mkdir(loc_OutputFiles)
        loc_OutputFiles += '/TrainingData/'
        if os.path.isdir(loc_OutputFiles) == False:
            os.mkdir(loc_OutputFiles)

        Y_Sims_PctID = gzip.open(loc_OutputFiles + 'Y_Sims_PctID.csv.gz', 'w')
        X_PctID = gzip.open(loc_OutputFiles + 'X_PctID.csv.gz','w')
        X_AvgB62 = gzip.open(loc_OutputFiles + 'X_AvgB62.csv.gz', 'w')
        X_PctID_Smooth3 = gzip.open(loc_OutputFiles + 'X_PctID_Smooth3.csv.gz', 'w')
        X_AvgB62_Smooth3 = gzip.open(loc_OutputFiles + 'X_AvgB62_Smooth3.csv.gz', 'w')
        
        for ID, aln in motifalignments.items():
            if pd.isnull(aln['EScoreOverlap']) == False:
                count += 1 #ID passed QC/Inclusion criteria 
                IDs.append(ID)
                uIDs.add(ID[0])
                uIDs.add(ID[1])
                
                #1) Parse the Y-info
                if count == 1:
                    h = ['MID_x', 'MID_y', 'EScoreOverlap', 'EClass','PctID_L', 'PctID_S', 'ArrayLenDifference', 'MultiAlnFlag']
                    Y_Sims_PctID.write(','.join(h) + '\n')
                oline = list(ID) 
                for col in ['EScoreOverlap', 'EClass','PctID_L', 'PctID_S', 'ArrayLenDifference', 'MultiAlnFlag']:
                    oline.append(aln[col])
                Y_Sims_PctID.write(','.join(map(str, oline)) + '\n')

                #2) Parse the X matrices
                for filehandle, dictID in zip([X_PctID, X_AvgB62, X_PctID_Smooth3, X_AvgB62_Smooth3], 
                                          ['ByPos.PctID', 'ByPos.AvgB62', 'ByPos.PctID.Smooth3', 'ByPos.AvgB62.Smooth3']):
                    if count == 1:
                        h = ['MID_x', 'MID_y'] + ['p' + str(x + 1) for x in range(len(aln['ByPos.PctID']))]
                        filehandle.write(','.join(h) + '\n')
                    oline = list(ID) + map(str, aln[dictID])
                    filehandle.write(','.join(oline) + '\n')
                    
        #Close Files
        for x in [X_PctID, X_AvgB62, X_PctID_Smooth3, X_AvgB62_Smooth3, Y_Sims_PctID]:
            x.close()

        #Calculate Testing folds
        with open(loc_OutputFiles + 'CVTestIndicies_i0.txt', 'w') as outf:
            for uID in uIDs:
                present_0 = []
                for i, ID in enumerate(IDs):
                    present_0.append(i)
                oline = [uID] + present_0
                outf.write('\t'.join(map(str, oline)) + '\n')

In [ ]:
CreateSRTrainingFrames(motifalignments= MotifAlignments, loc_OutputFiles='Example_HDTrainingFrames/')

## (B) Example of Scoring TF pairs using an existing model

In [ ]:
#Read SR Model
HD_SRModel = srpred.ReadSRModel('../SRModels/F223_1.97d.json')

SR_Scores_i = []
SR_Scores = []

#Score Motif Alignments
for pair, sr_alignment in MotifAlignments.items():
    SR_Score, SR_Class = srpred.ScoreAlignmentResult(resultDict=sr_alignment, scoreDict=HD_SRModel)
    SR_Scores_i.append(pair)
    SR_Scores.append([sr_alignment['PctID_L'], sr_alignment['EScoreOverlap'], SR_Score, SR_Class])
SR_Scores = pd.DataFrame(SR_Scores, columns = ['AA %ID','EScoreOverlap', 'SR_Score', 'SR_Class'])
SR_Scores.index = pd.MultiIndex.from_tuples(SR_Scores_i)
#Derive true TF Similarity Labels (Class)
SR_Scores['Class'] = 'Amb'
SR_Scores.loc[SR_Scores['EScoreOverlap'] >= Homeodomain_ReplicateThreshold, 'Class'] = 'HSim'
SR_Scores.loc[SR_Scores['EScoreOverlap'] < 0.2, 'Class'] = 'Dis'
SR_Scores.sort_values('SR_Score', ascending=False)